In [121]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [122]:
from mlxtend.regressor import StackingCVRegressor
from skopt import BayesSearchCV
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
import pandas as pd
import os
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit
from pmdarima.model_selection import train_test_split
import optuna
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import joblib

path_data = here("./data")
os.chdir(path_data)
data_iu_analysis = pd.read_parquet("data_iu_analysis.parquet")

In [123]:
train_size = round(data_iu_analysis.shape[0] * 0.8)
train_size

59

In [124]:
train, test = train_test_split(data_iu_analysis, test_size=train_size)

In [125]:
x_train = train.drop(columns="tournament_make")
x_test = test.drop(columns="tournament_make")
y_train = train["tournament_make"]
y_test = test["tournament_make"]

In [132]:
from lightgbm.sklearn import LGBMClassifier


params = {"max_depth": 3, "eta": 0.1, "objective": "binary", "eval_metric": "auc"}


# Create XGBoost model
model = LGBMClassifier(**params)

# Train XGBoost model
model.fit(x_train, y_train)

LGBMClassifier(eta=0.1, eval_metric='auc', max_depth=3, objective='binary')

In [127]:
# Train final XGBoost model using the best hyperparameters found by Optuna
best_params = study.best_params
best_model = xgb.XGBRegressor(**best_params)
best_model.fit(x_train, y_train)

# Return the best model
best_model

ValueError: No trials are completed yet.

In [9]:
path_outputs = here("./outputs")
os.chdir(path_outputs)
joblib.dump(best_model, "model_iu_bball.jlib")

['model_iu_bball.jlib']

In [10]:
prob = best_model.predict(x_test)

In [11]:
from sklearn.metrics import f1_score


def find_best_threshold(y_true, y_pred_prob):
    """
    Find the best threshold to use for converting predicted probabilities to binary predictions,
    using the F1 score as the metric to optimize for.

    :param y_true: true labels, array-like of shape (n_samples,)
    :param y_pred_prob: predicted probabilities, array-like of shape (n_samples,)
    :return: best threshold value
    """
    thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    best_f1 = 0
    best_threshold = 0
    for threshold in thresholds:
        y_pred = (y_pred_prob >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    return best_threshold

In [12]:
threshold = find_best_threshold(y_true=y_test, y_pred_prob=prob)
threshold

0.1

In [13]:
prob_threshold = np.where(prob > threshold, 1, 0)

In [14]:
from sklearn.metrics import balanced_accuracy_score, f1_score

In [15]:
balanced_accuracy = balanced_accuracy_score(y_true=y_test, y_pred=prob_threshold)
balanced_accuracy

0.6929411764705883

In [16]:
f1_accuracy = f1_score(y_true=y_test, y_pred=prob_threshold)
f1_accuracy

0.7272727272727272